# PEC0 - Manuel Nevado Fabián

### Enunciado
Train a deep MLP on the MNIST dataset (you can load it using tf.keras.​data⁠sets.mnist.load_data()). See if you can get over 98% accuracy by manually tuning the hyperparameters. Try searching for the optimal learning rate by using the approach presented in this chapter (i.e., by growing the learning rate exponentially, plotting the loss, and finding the point where the loss shoots up). Next, try tuning the hyperparameters using Keras Tuner with all the bells and whistles—save checkpoints, use early stopping, and plot learning curves using TensorBoard.

Este ejercicio aparece resuelto en el github por lo cual vamos a perdir que se realicen las siguientes modificaciones respecto a lo que se indica en el libro:

1) Hay que construir redes con al menos 3 y 4 capas ocultas.
2) Tanto para la red de 3 como para la de 4 capas ocultas hay que crear dos versiones con diferentes números de neuronas en las capas ocultas y comparar los resultados. Aquí se utilizará el factor de LearningRate que se utiliza en el notebook del libro.
3) Para las redes con 3 y 4 capas que hayan obtenido los mejores resultados en el apartado anterior cambiar el factor de LearningRate y comparar los resultados con los obtenidos en el apartado 2).

In [1]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
%load_ext tensorboard

2025-12-03 20:07:25.819090: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 20:07:25.858187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 20:07:26.750706: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Lo primero es descargar el dataset

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

Aqui lo que se va a ahcer es de las 60000 images totales se seleccionan las 5000 primeras para validacion y las demas se dejan de entreanmiento. Tambien se convierte los colores de los pixeles de enteros a flotantes para acotarlos entre 0 y 1 y que se procesen mas facilmente por las neuronas.

In [3]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

Aqui lo que vamos a hacer es encontrar la tasa de aprendizaje optima para este problema de regresion, a su vez, lo que vamos a hacer es guardar el valor de la funcion de perdida por cada epoca

Aqui lo que se hace es settear la semilla para que el experimento sea reproducible

In [4]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

Creamos los modelos

In [5]:
models_list = list()

model_3_A = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
models_list.append(model_3_A)

model_3_B = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(250, activation="relu"),
    tf.keras.layers.Dense(80, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
models_list.append(model_3_B)

model_4_A = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(350, activation="relu"),
    tf.keras.layers.Dense(275, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
models_list.append(model_4_A)

model_4_B = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(400, activation="relu"),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
models_list.append(model_4_B)

/home/manuel/uned/.venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
E0000 00:00:1764788847.538565   18654 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1764788847.543559   18654 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
for model in models_list:
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-1)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
    run_index = 1 # increment this at every run
    run_logdir = Path() / f"{model.name}_my_mnist_logs" / "run_{:03d}".format(run_index)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20)
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_mnist_model.keras", save_best_only=True)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

    history = model.fit(X_train, y_train, epochs=100,
                        validation_data=(X_valid, y_valid),
                        callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])
    model = tf.keras.models.load_model("my_mnist_model.keras") # rollback to best model
    model.evaluate(X_test, y_test)

Epoch 1/100
 258/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 982us/step - accuracy: 0.9983 - loss: 0.0063

In [10]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_logs